In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Linear Regression

Let's take a moment to summarize what we learned last class. Today we will use a simulated example where we actually know what the true linear relation is:

In [ ]:
n=10
x = norm.rvs(size = n)*10
x

In [ ]:
epsilon = norm.rvs(size=n)*15
y = 10 - 3*x + epsilon
y

Note that our error terms are the epsilon.


In [ ]:
plt.plot(x, y, 'b.')

We can compute the estimates for $\beta_0$ and $\beta_1$ via (note that there is a shortcut for this in scipy and also one in numpy):

In [ ]:
xbar = np.mean(x)
ybar = np.mean(y)

Sxy = sum( [ (x[i] - xbar)*(y[i] - ybar) for i in range(n) ])
Sxx = sum( [ (x[i] - xbar)**2 for i in range(n) ])

beta1 = Sxy / Sxx

beta0 = ybar - beta1*xbar

beta0, beta1

In [ ]:
plt.plot(x, y, 'b.')
xx = np.linspace(-15, 15, 100)
yy = beta0 + beta1*xx
plt.plot(xx, yy, 'b-')

We see our estimates are close, but not exact. 

We found formulas for the variances of the estimates for $\beta_0$ and $\beta_1$ in termss of the population variance of the error $\sigma^2$:

$$ V(\hat{\beta}_1) = \frac{\sigma^2}{S_{xx}}, \quad \mbox{and}\quad V(\hat{\beta_0}) = \frac{\sigma^2 \sum x_i^2}{n S_{xx}} $$

We actually know $\sigma$ because it is coded above in the definition of epsilon. However in general it is very unlikely we will know it and in pratice we will have to estimate it from the difference between the predicted $Y$ values:

$$ \hat{Y}_i = \hat{\beta}_0 + \hat{\beta}_1 x_i $$ 

and the actual $Y_i$. 

In [ ]:
S = np.sqrt( sum( [ (y[i] - (beta0 + beta1*x[i]) )**2 for i in range(n) ]) / (n-2) )
S

$S^2$ is an unbiased estimator for $\sigma^2$, and recall that in general we would want to divide by $n-d$ where $d$ is the number of parameters in our model. Also its worth noting that 8 degrees of freedom we have here is quite small and we expect significant error in our estimates.

This then means that in using the variances for $\beta_0$ and $\beta_1$ to construct confidence intervals or hypothesis tests we will need to use the t-distribution.

In [ ]:
# Computing the (estimates) of the variances of the (estimates)

Vbeta1 = S**2 / Sxx
Vbeta0 = S**2 * sum( [x[i]**2 for i in range(n) ] ) / n / Sxx

Vbeta0, Vbeta1

In [ ]:
# The standard deviations

np.sqrt(Vbeta0), np.sqrt(Vbeta1)

Significant variation predicted for both. Certainly a big change from Tuesday's lesson is that in this example the sample size is far to small for tight conclusions.

Let's build confidence intervals of 99% significance around each estimate:

In [ ]:
from scipy.stats import t

In [ ]:
# Recall you need to input the degrees of freedom

t_alpha = t.ppf(0.995, n-2) 
t_alpha

In [ ]:
# beta0 confidence interval

beta0 - t_alpha*np.sqrt(Vbeta0), beta0 + t_alpha*np.sqrt(Vbeta0)

In [ ]:
# beta1 confidence interval

beta1 - t_alpha * np.sqrt(Vbeta1), beta1 + t_alpha*np.sqrt(Vbeta1)

Note a couple of things:  

- We expect to 99% of the time end up with a confidence interval that contains the true value of beta0 and beta1 that we found above. 
- We could for example use the computation we just did to say that we can conclude that $\beta_1 < 1$. 
- We would not be able to conclude that the true value of $\beta_0 \neq 0$.

Finally recall that we had a formula for the Covariance:

$$ \mbox{Cov}(\hat{\beta}_0, \hat{\beta}_1) = - \frac{\sigma^2 \bar{x}}{S_{xx}} $$

## Estimates

Of course the way that we will typically use our linear model of the data is to make predictions. For example here we can try and predict what the value for $Y$ will be when $x = 0$ or $x=20$.

Before we do this though, check the graph above. Note one thing we expect:  The estimate should be more accurate the closer the x value is to $\bar{x}$.  I.e. we expect the prediction for $x=0$ to be much more accurate than the one for $x=20$ and our confidence interval should end up reflecting that.

In [ ]:
# The two predictions:

beta0 + beta1*0, beta0 + beta1*20

But what we really want to give is an interval of $Y$ values in which we are 99% confident (or whatever confidence) for a given $x$. To do that we need to understand the variation of the error $$ Y - \hat{Y}$$ for a given $x$.  The trick is that we are treating $x$ as a known fixed value $Y$ as a random variable. 

First though note that the expected value of the error is:

$$ E( Y - \hat{Y} | x ) = E( \epsilon) $$ 

and one of our assumptions is that $\epsilon$ is normally distributed with mean 0. It is interesting to consider what happens when this is false, however for now let's proceed with it.

Then the variance, in order to compute this we do need to check that $Y$ and $\hat{Y}$ are uncorrelated for each $x$. this is true because the $Y$ is not used in the computation of $\hat{Y}$. We also need that to recognize that 

$$ V(Y| x) = \sigma^2 $$

I.e. that for a fixed $x$ the variation in $Y$ is entirely due to the variation in $\epsilon$:

$$ V(Y - \hat{Y} | x ) = V(Y | x) + V(\hat{Y} | x) = \sigma^2 + V(\hat{\beta}_0 + \hat{\beta}_1 x) $$ 

Note that we can't just distribute the variation over the linear model because the $\beta_j$ are correlated. Also note why we couldn't just use $\sigma^2$ for the variation of the error because it is the error from the true model, whereas our $\hat{\beta}_j$ are estimates and so themselves have some variation.

$$ V(\hat{\beta}_0 + \hat{\beta}_1 x ) = V(\hat{\beta}_0) + x^2 V(\hat{\beta}_1) + 2 x \mbox{Cov}(\hat{\beta}_0, \hat{\beta}_1 ) $$

$$ = \frac{\sigma^2 \sum x_i^2}{n S_{xx}} + \frac{\sigma^2 x^2}{S_{xx}} - \frac{2 \sigma^2 x \bar{x} }{ S_{xx}} $$

$$ = \frac{\sigma^2 \sum x_i^2}{n S_{xx}} + \frac{\sigma^2 (x - \bar{x})^2}{S_{xx}} - \frac{\sigma^2 \bar{x}^2}{ S_{xx} } $$

$$ = \frac{\sigma^2 \sum x_i ( x_i - \bar{x} ) }{n S_{xx}} + \frac{\sigma^2 (x - \bar{x})^2}{S_{xx}} $$

$$ = \frac{ \sigma^2 \sum (x_i - \bar{x})^2 }{n S_{xx}} + \frac{\sigma^2 \sum \bar{x} (x_i - \bar{x} ) }{n S_{xx}} 
+ \frac{\sigma^2 (x - \bar{x})^2}{S_{xx}} $$

$$ = \frac{\sigma^2}{n} + \frac{ \sigma^2 (x - \bar{x})^2}{S_{xx}} $$

Putting it all together than we have:

$$ V(Y - \hat{Y} | x) = \sigma^2 \left[ 1 + \frac{1}{n} + \frac{ (x - \bar{x})^2}{S_{xx}} \right] $$

- What happens as $n$ gets large?

- What happens as the spread of the $x_i$ increases?

- What happens as the variance of the error decreases?

- What happens as $x$ moves closer or further from $\bar{x}$?

From here it is then straightforward to build confidence intervals around our estimates: 

For $x=0$ in our example we find for 90% confidence.

In [ ]:
t_alpha = t.ppf(0.95, n-2)
t_alpha

In [ ]:
plt.plot(x, y, 'b.')
xx = np.linspace(-25, 25, 100)
yy = beta0 + beta1*xx
plt.plot(xx, yy, 'b-')

In [ ]:
V = S * np.sqrt(1 + 1/n + (0 - xbar)**2 / Sxx )

beta0 + beta1*0 - t_alpha*V, beta0 + beta1*0 + t_alpha*V

In [ ]:
V = S * np.sqrt(1 + 1/n + (20 - xbar)**2 / Sxx )

beta0 + beta1*20 - t_alpha*V, beta0 + beta1*20 + t_alpha*V

## The effect of $n$

These are pretty useless intervals. Note the problem is that $n$ is not very big, this affects $t_\alpha$ as well as the variance. 

Let's do the experiment again with a much large $n$:

In [ ]:
n=200
x = norm.rvs(size = n)*10
epsilon = norm.rvs(size=n)*15
y = 10 - 3*x + epsilon

xbar = np.mean(x)
ybar = np.mean(y)

Sxy = sum( [ (x[i] - xbar)*(y[i] - ybar) for i in range(n) ])
Sxx = sum( [ (x[i] - xbar)**2 for i in range(n) ])

beta1 = Sxy / Sxx

beta0 = ybar - beta1*xbar

beta0, beta1

In [ ]:
plt.plot(x, y, 'b.')
xx = np.linspace(-25, 25, 100)
yy = beta0 + beta1*xx
plt.plot(xx, yy, 'b-')

In [ ]:
S = np.sqrt( sum( [ (y[i] - (beta0 + beta1*x[i]) )**2 for i in range(n) ]) / (n-2) )

In [ ]:
V = S * np.sqrt(1 + 1/n + (0 - xbar)**2 / Sxx )

beta0 + beta1*0 - t_alpha*V, beta0 + beta1*0 + t_alpha*V

In [ ]:
V = S * np.sqrt(1 + 1/n + (20 - xbar)**2 / Sxx )

beta0 + beta1*20 - t_alpha*V, beta0 + beta1*20 + t_alpha*V